# 🧹 Módulo de Limpieza Avanzada de Datos

## 📦 Instalación de Dependencias

In [ ]:
# Instalar dependencias necesarias
!pip install pandas numpy scikit-learn matplotlib seaborn scipy -q

## 📚 Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Optional, Tuple
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# 🔧 PARTE 1: Módulo de Limpieza

## 1.1 Clase AnalizadorDatos

In [ ]:
class AnalizadorDatos:
    """
    Clase para identificar y analizar tipos de variables y datos faltantes.
    """

    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.reporte = {}

    def identificar_tipos_variables(self) -> Dict[str, Dict]:
        """Identifica tipos de variables con análisis estadístico detallado."""
        print("\n" + "="*70)
        print("🔍 ANÁLISIS DE TIPOS DE VARIABLES")
        print("="*70)

        clasificacion = {
            'numericas_continuas': [],
            'numericas_discretas': [],
            'categoricas_nominales': [],
            'categoricas_ordinales': [],
            'fechas': [],
            'texto': [],
            'booleanas': []
        }

        for col in self.df.columns:
            tipo_detectado = self._detectar_tipo_variable(col)
            clasificacion[tipo_detectado].append(col)

        # Imprimir reporte
        for tipo, columnas in clasificacion.items():
            if columnas:
                print(f"\n📊 {tipo.upper().replace('_', ' ')}: {len(columnas)}")
                for col in columnas:
                    stats = self._estadisticas_columna(col)
                    print(f"   • {col}: {stats}")

        self.reporte['clasificacion'] = clasificacion
        return clasificacion

    def _detectar_tipo_variable(self, col: str) -> str:
        """Detecta el tipo de variable usando heurísticas estadísticas."""
        serie = self.df[col]

        if self._es_fecha(serie):
            return 'fechas'

        if serie.dtype == bool or set(serie.dropna().unique()).issubset({0, 1, True, False, 'True', 'False', 'true', 'false'}):
            return 'booleanas'

        if pd.api.types.is_numeric_dtype(serie):
            valores_unicos = serie.nunique()
            n_total = len(serie.dropna())

            if valores_unicos < 10 or valores_unicos / n_total < 0.05:
                return 'numericas_discretas'
            else:
                if serie.dtype in ['int64', 'int32'] and valores_unicos < n_total * 0.5:
                    return 'numericas_discretas'
                return 'numericas_continuas'
        else:
            valores_unicos = serie.nunique()
            if valores_unicos < 20:
                return 'categoricas_nominales'
            elif valores_unicos < 50:
                return 'categoricas_ordinales'
            else:
                return 'texto'

    def _es_fecha(self, serie: pd.Series) -> bool:
        """Verifica si una columna contiene fechas."""
        if pd.api.types.is_datetime64_any_dtype(serie):
            return True
        try:
            muestra = serie.dropna().head(100)
            if len(muestra) > 0:
                pd.to_datetime(muestra, errors='coerce')
                return True
        except:
            pass
        return False

    def _estadisticas_columna(self, col: str) -> str:
        """Genera estadísticas descriptivas de una columna."""
        serie = self.df[col]
        n_valores = len(serie)
        n_nulos = serie.isna().sum()
        n_unicos = serie.nunique()
        pct_nulos = (n_nulos / n_valores) * 100

        if pd.api.types.is_numeric_dtype(serie):
            return f"Únicos={n_unicos}, Nulos={n_nulos} ({pct_nulos:.1f}%), Rango=[{serie.min():.2f}, {serie.max():.2f}]"
        else:
            return f"Únicos={n_unicos}, Nulos={n_nulos} ({pct_nulos:.1f}%)"

    def analizar_datos_faltantes(self) -> pd.DataFrame:
        """Analiza patrones de datos faltantes usando métodos estadísticos."""
        print("\n" + "="*70)
        print("📉 ANÁLISIS DE DATOS FALTANTES")
        print("="*70)

        analisis = []

        for col in self.df.columns:
            n_nulos = self.df[col].isna().sum()
            pct_nulos = (n_nulos / len(self.df)) * 100

            if n_nulos > 0:
                patron = self._detectar_patron_missingness(col)

                analisis.append({
                    'columna': col,
                    'n_nulos': n_nulos,
                    'pct_nulos': pct_nulos,
                    'patron': patron,
                    'tipo': self._detectar_tipo_variable(col)
                })

        df_analisis = pd.DataFrame(analisis).sort_values('pct_nulos', ascending=False)

        if len(df_analisis) > 0:
            print("\n📋 Resumen de Datos Faltantes:")
            print(df_analisis.to_string(index=False))
            print(f"\n🔬 Patrón de Missingness: {self._test_mcar()}")
        else:
            print("\n✅ No se detectaron datos faltantes")

        self.reporte['datos_faltantes'] = df_analisis
        return df_analisis

    def _detectar_patron_missingness(self, col: str) -> str:
        """Detecta el patrón de datos faltantes (MCAR, MAR, MNAR)."""
        mascara_nulos = self.df[col].isna()
        correlaciones = []

        for otra_col in self.df.columns:
            if otra_col != col and pd.api.types.is_numeric_dtype(self.df[otra_col]):
                corr = np.corrcoef(mascara_nulos, self.df[otra_col].fillna(0))[0, 1]
                if abs(corr) > 0.3:
                    correlaciones.append((otra_col, corr))

        if len(correlaciones) > 0:
            return "MAR (posiblemente)"
        else:
            return "MCAR (posiblemente)"

    def _test_mcar(self) -> str:
        """Test simplificado de Little para MCAR."""
        nulos_por_fila = self.df.isna().sum(axis=1)
        varianza = nulos_por_fila.var()

        if varianza < 1:
            return "Probable (baja varianza en distribución de nulos)"
        else:
            return "Improbable (alta varianza sugiere patrón estructurado)"

    def generar_reporte_completo(self) -> Dict:
        """Genera un reporte completo del análisis."""
        self.identificar_tipos_variables()
        self.analizar_datos_faltantes()

        print("\n" + "="*70)
        print("📊 RESUMEN GENERAL")
        print("="*70)
        print(f"Filas: {len(self.df):,}")
        print(f"Columnas: {len(self.df.columns)}")
        print(f"Memoria: {self.df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

        return self.reporte

## 1.2 Clase ImputadorAvanzado

In [ ]:
class ImputadorAvanzado:
    """
    Clase para imputación de datos usando métodos estadísticos avanzados.
    """

    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.df_original = df.copy()

    def imputar_knn(self, columnas: List[str], n_neighbors: int = 5) -> pd.DataFrame:
        """Imputa valores usando K-Nearest Neighbors."""
        from sklearn.impute import KNNImputer

        print("\n" + "="*70)
        print(f"🔧 IMPUTACIÓN KNN (k={n_neighbors})")
        print("="*70)

        df_numerico = self.df.select_dtypes(include=[np.number])
        imputer = KNNImputer(n_neighbors=n_neighbors, weights='distance')
        datos_imputados = imputer.fit_transform(df_numerico)

        df_resultado = self.df.copy()
        df_resultado[df_numerico.columns] = datos_imputados

        for col in columnas:
            if col in df_numerico.columns:
                n_imputados = self.df_original[col].isna().sum()
                if n_imputados > 0:
                    valor_medio_imputado = df_resultado.loc[self.df_original[col].isna(), col].mean()
                    print(f"✅ {col}: {n_imputados} valores imputados (media imputada: {valor_medio_imputado:.2f})")

        self.df = df_resultado
        return df_resultado

    def imputar_mice(self, columnas: List[str], max_iter: int = 10) -> pd.DataFrame:
        """Imputa valores usando MICE (Multivariate Imputation by Chained Equations)."""
        from sklearn.experimental import enable_iterative_imputer
        from sklearn.impute import IterativeImputer

        print("\n" + "="*70)
        print(f"🔧 IMPUTACIÓN MICE (max_iter={max_iter})")
        print("="*70)

        df_numerico = self.df.select_dtypes(include=[np.number])
        imputer = IterativeImputer(max_iter=max_iter, random_state=42, verbose=0)
        datos_imputados = imputer.fit_transform(df_numerico)

        df_resultado = self.df.copy()
        df_resultado[df_numerico.columns] = datos_imputados

        for col in columnas:
            if col in df_numerico.columns:
                n_imputados = self.df_original[col].isna().sum()
                if n_imputados > 0:
                    valor_medio_imputado = df_resultado.loc[self.df_original[col].isna(), col].mean()
                    print(f"✅ {col}: {n_imputados} valores imputados (media imputada: {valor_medio_imputado:.2f})")

        self.df = df_resultado
        return df_resultado

    def imputar_interpolacion(self, columnas: List[str], metodo: str = 'linear') -> pd.DataFrame:
        """Imputa valores usando interpolación."""
        print("\n" + "="*70)
        print(f"🔧 IMPUTACIÓN POR INTERPOLACIÓN ({metodo})")
        print("="*70)

        df_resultado = self.df.copy()

        for col in columnas:
            if col in df_resultado.columns:
                n_imputados = df_resultado[col].isna().sum()
                if n_imputados > 0:
                    df_resultado[col] = df_resultado[col].interpolate(method=metodo, limit_direction='both')
                    print(f"✅ {col}: {n_imputados} valores imputados por interpolación {metodo}")

        self.df = df_resultado
        return df_resultado

    def imputar_regresion(self, columnas: List[str]) -> pd.DataFrame:
        """Imputa valores usando regresión lineal multivariable."""
        from sklearn.linear_model import LinearRegression

        print("\n" + "="*70)
        print("🔧 IMPUTACIÓN POR REGRESIÓN LINEAL")
        print("="*70)

        df_resultado = self.df.copy()

        for col in columnas:
            if col in df_resultado.columns and pd.api.types.is_numeric_dtype(df_resultado[col]):
                n_imputados = df_resultado[col].isna().sum()

                if n_imputados > 0:
                    mask_completos = df_resultado[col].notna()
                    features = df_resultado.select_dtypes(include=[np.number]).columns.drop(col)
                    features_sin_nulos = [f for f in features if df_resultado[f].isna().sum() == 0]

                    if len(features_sin_nulos) > 0:
                        X_train = df_resultado.loc[mask_completos, features_sin_nulos]
                        y_train = df_resultado.loc[mask_completos, col]
                        X_pred = df_resultado.loc[~mask_completos, features_sin_nulos]

                        modelo = LinearRegression()
                        modelo.fit(X_train, y_train)
                        y_pred = modelo.predict(X_pred)
                        df_resultado.loc[~mask_completos, col] = y_pred

                        print(f"✅ {col}: {n_imputados} valores imputados por regresión (R² = {modelo.score(X_train, y_train):.3f})")
                    else:
                        print(f"⚠️ {col}: No hay suficientes features para regresión")

        self.df = df_resultado
        return df_resultado

    def eliminar_categoricos_nulos(self, umbral_porcentaje: float = 50.0) -> pd.DataFrame:
        """Elimina filas con valores nulos en variables categóricas."""
        print("\n" + "="*70)
        print("🗑️  ELIMINACIÓN DE NULOS EN VARIABLES CATEGÓRICAS")
        print("="*70)

        df_resultado = self.df.copy()
        n_inicial = len(df_resultado)
        cols_categoricas = df_resultado.select_dtypes(include=['object', 'category']).columns
        columnas_eliminadas = []

        for col in cols_categoricas:
            pct_nulos = (df_resultado[col].isna().sum() / len(df_resultado)) * 100

            if pct_nulos > umbral_porcentaje:
                df_resultado = df_resultado.drop(columns=[col])
                columnas_eliminadas.append(col)
                print(f"🗑️  Columna '{col}' eliminada ({pct_nulos:.1f}% nulos)")
            elif pct_nulos > 0:
                n_nulos = df_resultado[col].isna().sum()
                df_resultado = df_resultado.dropna(subset=[col])
                print(f"✅ '{col}': {n_nulos} filas eliminadas ({pct_nulos:.1f}% nulos)")

        n_final = len(df_resultado)
        print(f"\n📊 Filas totales eliminadas: {n_inicial - n_final} ({((n_inicial - n_final) / n_inicial * 100):.1f}%)")

        self.df = df_resultado
        return df_resultado

## 1.3 Clase CorreccionFormatos

In [ ]:
class CorreccionFormatos:
    """Clase para corrección de formatos, especialmente fechas."""

    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()

    def corregir_fechas(self, columnas: Optional[List[str]] = None,
                       formato: Optional[str] = None) -> pd.DataFrame:
        """Detecta y corrige formatos de fecha."""
        print("\n" + "="*70)
        print("📅 CORRECCIÓN DE FORMATOS DE FECHA")
        print("="*70)

        df_resultado = self.df.copy()

        if columnas is None:
            columnas = []
            for col in df_resultado.columns:
                if self._es_posible_fecha(df_resultado[col]):
                    columnas.append(col)

        for col in columnas:
            try:
                if formato:
                    df_resultado[col] = pd.to_datetime(df_resultado[col], format=formato, errors='coerce')
                else:
                    df_resultado[col] = pd.to_datetime(df_resultado[col], infer_datetime_format=True, errors='coerce')

                n_convertidos = df_resultado[col].notna().sum()
                n_fallidos = df_resultado[col].isna().sum()
                print(f"✅ '{col}': {n_convertidos} fechas convertidas, {n_fallidos} fallos")

                if n_convertidos > 0:
                    min_fecha = df_resultado[col].min()
                    max_fecha = df_resultado[col].max()
                    print(f"   Rango: {min_fecha} a {max_fecha}")

            except Exception as e:
                print(f"❌ Error en '{col}': {str(e)}")

        self.df = df_resultado
        return df_resultado

    def _es_posible_fecha(self, serie: pd.Series) -> bool:
        """Verifica si una columna podría contener fechas."""
        if pd.api.types.is_datetime64_any_dtype(serie):
            return True
        if pd.api.types.is_numeric_dtype(serie):
            return False

        muestra = serie.dropna().head(50)
        if len(muestra) == 0:
            return False

        try:
            convertidos = pd.to_datetime(muestra, errors='coerce')
            pct_exitoso = convertidos.notna().sum() / len(muestra)
            return pct_exitoso > 0.5
        except:
            return False

## 1.4 Clase LimpiadorCompleto

In [ ]:
class LimpiadorCompleto:
    """Clase orquestadora que combina todos los métodos de limpieza."""

    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.df_original = df.copy()
        self.historial = []

    def pipeline_completo(self, metodo_imputacion: str = 'knn',
                         eliminar_nulos_categoricos: bool = True,
                         corregir_fechas: bool = True,
                         **kwargs) -> pd.DataFrame:
        """Ejecuta el pipeline completo de limpieza."""
        print("\n" + "="*70)
        print("🚀 INICIANDO PIPELINE DE LIMPIEZA COMPLETO")
        print("="*70)

        analizador = AnalizadorDatos(self.df)
        reporte = analizador.generar_reporte_completo()
        clasificacion = reporte['clasificacion']

        if corregir_fechas and len(clasificacion['fechas']) > 0:
            corrector = CorreccionFormatos(self.df)
            self.df = corrector.corregir_fechas()
            self.historial.append("Fechas corregidas")

        cols_numericas = clasificacion['numericas_continuas'] + clasificacion['numericas_discretas']
        cols_con_nulos = [col for col in cols_numericas if self.df[col].isna().sum() > 0]

        if len(cols_con_nulos) > 0:
            imputador = ImputadorAvanzado(self.df)

            if metodo_imputacion == 'knn':
                n_neighbors = kwargs.get('n_neighbors', 5)
                self.df = imputador.imputar_knn(cols_con_nulos, n_neighbors=n_neighbors)
            elif metodo_imputacion == 'mice':
                max_iter = kwargs.get('max_iter', 10)
                self.df = imputador.imputar_mice(cols_con_nulos, max_iter=max_iter)
            elif metodo_imputacion == 'interpolacion':
                metodo = kwargs.get('metodo_interpolacion', 'linear')
                self.df = imputador.imputar_interpolacion(cols_con_nulos, metodo=metodo)
            elif metodo_imputacion == 'regresion':
                self.df = imputador.imputar_regresion(cols_con_nulos)

            self.historial.append(f"Imputación {metodo_imputacion} aplicada")

        if eliminar_nulos_categoricos:
            imputador = ImputadorAvanzado(self.df)
            umbral = kwargs.get('umbral_categoricos', 50.0)
            self.df = imputador.eliminar_categoricos_nulos(umbral_porcentaje=umbral)
            self.historial.append("Nulos categóricos eliminados")

        self._resumen_final()
        return self.df

    def _resumen_final(self):
        """Imprime resumen final de la limpieza."""
        print("\n" + "="*70)
        print("📊 RESUMEN FINAL DE LIMPIEZA")
        print("="*70)
        print(f"\n🔸 Filas originales: {len(self.df_original):,}")
        print(f"🔸 Filas finales: {len(self.df):,}")
        print(f"🔸 Filas eliminadas: {len(self.df_original) - len(self.df):,}")
        print(f"\n🔸 Columnas originales: {len(self.df_original.columns)}")
        print(f"🔸 Columnas finales: {len(self.df.columns)}")

        nulos_originales = self.df_original.isna().sum().sum()
        nulos_finales = self.df.isna().sum().sum()
        print(f"\n🔸 Nulos originales: {nulos_originales:,}")
        print(f"🔸 Nulos finales: {nulos_finales:,}")
        print(f"🔸 Reducción de nulos: {((nulos_originales - nulos_finales) / max(nulos_originales, 1) * 100):.1f}%")
        print("\n✅ Pipeline de limpieza completado")

# 📊 PARTE 2: Tests Estadísticos para Selección de Método

## 2.1 Clase EvaluadorMetodos

In [ ]:
class EvaluadorMetodos:
    """
    Clase para evaluar y comparar métodos de imputación usando tests estadísticos.
    """

    def __init__(self, df_original: pd.DataFrame):
        """Inicializa con el dataset original (con valores completos si es posible)."""
        self.df_original = df_original.copy()
        self.resultados = {}

    def introducir_nulos_controlados(self, columnas: List[str], porcentaje: float = 20.0) -> pd.DataFrame:
        """
        Introduce valores nulos de forma controlada para evaluar métodos.

        Args:
            columnas: Columnas donde introducir nulos
            porcentaje: Porcentaje de valores a eliminar
        """
        df_con_nulos = self.df_original.copy()

        np.random.seed(42)

        for col in columnas:
            n_nulos = int(len(df_con_nulos) * porcentaje / 100)
            indices = np.random.choice(df_con_nulos.index, n_nulos, replace=False)
            df_con_nulos.loc[indices, col] = np.nan

        print(f"\n✅ Introducidos {porcentaje}% de nulos en {len(columnas)} columnas")
        return df_con_nulos

    def calcular_metricas_error(self, df_imputado: pd.DataFrame,
                               columnas: List[str],
                               mascara_nulos: pd.DataFrame) -> Dict:
        """
        Calcula métricas de error comparando valores imputados con originales.

        Returns:
            Diccionario con MAE, RMSE, MAPE, R²
        """
        from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

        metricas = {}

        for col in columnas:
            # Obtener valores originales y imputados solo donde había nulos
            valores_originales = self.df_original.loc[mascara_nulos[col], col]
            valores_imputados = df_imputado.loc[mascara_nulos[col], col]

            # Calcular métricas
            mae = mean_absolute_error(valores_originales, valores_imputados)
            rmse = np.sqrt(mean_squared_error(valores_originales, valores_imputados))

            # MAPE (Mean Absolute Percentage Error)
            mape = np.mean(np.abs((valores_originales - valores_imputados) / valores_originales)) * 100

            # R² Score
            r2 = r2_score(valores_originales, valores_imputados)

            # Bias (sesgo)
            bias = np.mean(valores_imputados - valores_originales)

            metricas[col] = {
                'MAE': mae,
                'RMSE': rmse,
                'MAPE': mape,
                'R2': r2,
                'Bias': bias,
                'n_valores': len(valores_originales)
            }

        return metricas

    def test_kolmogorov_smirnov(self, df_imputado: pd.DataFrame,
                                columnas: List[str]) -> Dict:
        """
        Test de Kolmogorov-Smirnov para comparar distribuciones.
        H0: Las distribuciones son iguales
        """
        from scipy.stats import ks_2samp

        resultados_ks = {}

        for col in columnas:
            valores_originales = self.df_original[col].dropna()
            valores_imputados = df_imputado[col].dropna()

            statistic, p_value = ks_2samp(valores_originales, valores_imputados)

            resultados_ks[col] = {
                'statistic': statistic,
                'p_value': p_value,
                'son_similares': p_value > 0.05  # No rechazamos H0
            }

        return resultados_ks

    def test_anderson_darling(self, df_imputado: pd.DataFrame,
                             columnas: List[str]) -> Dict:
        """
        Test de Anderson-Darling para normalidad.
        """
        from scipy.stats import anderson

        resultados_ad = {}

        for col in columnas:
            valores_imputados = df_imputado[col].dropna()

            result = anderson(valores_imputados)

            resultados_ad[col] = {
                'statistic': result.statistic,
                'critical_values': result.critical_values,
                'significance_level': result.significance_level,
                'es_normal': result.statistic < result.critical_values[2]  # 5% nivel
            }

        return resultados_ad

    def comparar_metodos(self, columnas: List[str],
                        porcentaje_nulos: float = 20.0,
                        metodos: List[str] = None) -> pd.DataFrame:
        """
        Compara todos los métodos de imputación con tests estadísticos.

        Returns:
            DataFrame con comparación detallada
        """
        if metodos is None:
            metodos = ['knn', 'mice', 'regresion', 'interpolacion']

        print("\n" + "="*70)
        print("🔬 COMPARACIÓN ESTADÍSTICA DE MÉTODOS DE IMPUTACIÓN")
        print("="*70)

        # Introducir nulos controlados
        df_con_nulos = self.introducir_nulos_controlados(columnas, porcentaje_nulos)
        mascara_nulos = df_con_nulos[columnas].isna()

        comparacion = []

        for metodo in metodos:
            print(f"\n{'='*70}")
            print(f"Evaluando método: {metodo.upper()}")
            print(f"{'='*70}")

            try:
                # Aplicar imputación
                imputador = ImputadorAvanzado(df_con_nulos)

                if metodo == 'knn':
                    df_imputado = imputador.imputar_knn(columnas, n_neighbors=5)
                elif metodo == 'mice':
                    df_imputado = imputador.imputar_mice(columnas, max_iter=10)
                elif metodo == 'regresion':
                    df_imputado = imputador.imputar_regresion(columnas)
                elif metodo == 'interpolacion':
                    df_imputado = imputador.imputar_interpolacion(columnas, metodo='linear')

                # Calcular métricas de error
                metricas = self.calcular_metricas_error(df_imputado, columnas, mascara_nulos)

                # Tests estadísticos
                ks_test = self.test_kolmogorov_smirnov(df_imputado, columnas)

                # Agregar a comparación
                for col in columnas:
                    comparacion.append({
                        'Método': metodo.upper(),
                        'Columna': col,
                        'MAE': metricas[col]['MAE'],
                        'RMSE': metricas[col]['RMSE'],
                        'MAPE (%)': metricas[col]['MAPE'],
                        'R²': metricas[col]['R2'],
                        'Bias': metricas[col]['Bias'],
                        'KS p-value': ks_test[col]['p_value'],
                        'Dist. Similar': '✓' if ks_test[col]['son_similares'] else '✗'
                    })

            except Exception as e:
                print(f"❌ Error con método {metodo}: {str(e)}")

        df_comparacion = pd.DataFrame(comparacion)
        self.resultados['comparacion'] = df_comparacion

        return df_comparacion

    def recomendar_mejor_metodo(self, df_comparacion: pd.DataFrame) -> Dict:
        """
        Recomienda el mejor método basándose en múltiples criterios.
        """
        print("\n" + "="*70)
        print("🏆 RECOMENDACIÓN DE MEJOR MÉTODO")
        print("="*70)

        recomendaciones = {}

        for columna in df_comparacion['Columna'].unique():
            df_col = df_comparacion[df_comparacion['Columna'] == columna].copy()

            # Normalizar métricas (menor es mejor para MAE, RMSE, MAPE)
            df_col['Score_MAE'] = 1 - (df_col['MAE'] - df_col['MAE'].min()) / (df_col['MAE'].max() - df_col['MAE'].min() + 1e-10)
            df_col['Score_RMSE'] = 1 - (df_col['RMSE'] - df_col['RMSE'].min()) / (df_col['RMSE'].max() - df_col['RMSE'].min() + 1e-10)
            df_col['Score_MAPE'] = 1 - (df_col['MAPE (%)'] - df_col['MAPE (%)'].min()) / (df_col['MAPE (%)'].max() - df_col['MAPE (%)'].min() + 1e-10)

            # Mayor es mejor para R²
            df_col['Score_R2'] = (df_col['R²'] - df_col['R²'].min()) / (df_col['R²'].max() - df_col['R²'].min() + 1e-10)

            # Score compuesto (promedio ponderado)
            df_col['Score_Total'] = (
                df_col['Score_MAE'] * 0.3 +
                df_col['Score_RMSE'] * 0.3 +
                df_col['Score_MAPE'] * 0.2 +
                df_col['Score_R2'] * 0.2
            )

            mejor = df_col.loc[df_col['Score_Total'].idxmax()]

            recomendaciones[columna] = {
                'mejor_metodo': mejor['Método'],
                'score_total': mejor['Score_Total'],
                'mae': mejor['MAE'],
                'rmse': mejor['RMSE'],
                'mape': mejor['MAPE (%)'],
                'r2': mejor['R²']
            }

            print(f"\n📊 {columna}:")
            print(f"   🏅 Mejor método: {mejor['Método']}")
            print(f"   📈 Score total: {mejor['Score_Total']:.3f}")
            print(f"   📉 MAE: {mejor['MAE']:.4f}")
            print(f"   📉 RMSE: {mejor['RMSE']:.4f}")
            print(f"   📉 MAPE: {mejor['MAPE (%)']:.2f}%")
            print(f"   📈 R²: {mejor['R²']:.4f}")

        return recomendaciones

## 2.2 Funciones de Visualización

In [ ]:
def visualizar_comparacion_metodos(df_comparacion: pd.DataFrame):
    """
    Crea visualizaciones comparativas de los métodos de imputación.
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Comparación de Métodos de Imputación', fontsize=16, fontweight='bold')

    # 1. MAE por método
    ax1 = axes[0, 0]
    df_pivot = df_comparacion.pivot_table(values='MAE', index='Columna', columns='Método')
    df_pivot.plot(kind='bar', ax=ax1, width=0.8)
    ax1.set_title('MAE (Mean Absolute Error) - Menor es Mejor', fontweight='bold')
    ax1.set_ylabel('MAE')
    ax1.set_xlabel('Columna')
    ax1.legend(title='Método', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)

    # 2. RMSE por método
    ax2 = axes[0, 1]
    df_pivot = df_comparacion.pivot_table(values='RMSE', index='Columna', columns='Método')
    df_pivot.plot(kind='bar', ax=ax2, width=0.8)
    ax2.set_title('RMSE (Root Mean Squared Error) - Menor es Mejor', fontweight='bold')
    ax2.set_ylabel('RMSE')
    ax2.set_xlabel('Columna')
    ax2.legend(title='Método', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax2.grid(True, alpha=0.3)

    # 3. R² por método
    ax3 = axes[1, 0]
    df_pivot = df_comparacion.pivot_table(values='R²', index='Columna', columns='Método')
    df_pivot.plot(kind='bar', ax=ax3, width=0.8)
    ax3.set_title('R² Score - Mayor es Mejor', fontweight='bold')
    ax3.set_ylabel('R²')
    ax3.set_xlabel('Columna')
    ax3.legend(title='Método', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax3.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='Bueno (0.8)')
    ax3.grid(True, alpha=0.3)

    # 4. MAPE por método
    ax4 = axes[1, 1]
    df_pivot = df_comparacion.pivot_table(values='MAPE (%)', index='Columna', columns='Método')
    df_pivot.plot(kind='bar', ax=ax4, width=0.8)
    ax4.set_title('MAPE (%) - Menor es Mejor', fontweight='bold')
    ax4.set_ylabel('MAPE (%)')
    ax4.set_xlabel('Columna')
    ax4.legend(title='Método', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

def visualizar_distribucion_errores(df_original, df_imputado, columna, metodo, mascara_nulos):
    """
    Visualiza la distribución de errores para un método específico.
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle(f'Análisis de Errores: {metodo.upper()} - Columna: {columna}',
                 fontsize=14, fontweight='bold')

    # Obtener valores
    valores_originales = df_original.loc[mascara_nulos[columna], columna]
    valores_imputados = df_imputado.loc[mascara_nulos[columna], columna]
    errores = valores_imputados - valores_originales

    # 1. Scatter plot: Original vs Imputado
    ax1 = axes[0]
    ax1.scatter(valores_originales, valores_imputados, alpha=0.6, s=50)

    # Línea de referencia perfecta
    min_val = min(valores_originales.min(), valores_imputados.min())
    max_val = max(valores_originales.max(), valores_imputados.max())
    ax1.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfecta predicción')

    ax1.set_xlabel('Valores Originales', fontsize=11)
    ax1.set_ylabel('Valores Imputados', fontsize=11)
    ax1.set_title('Original vs Imputado', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Distribución de errores
    ax2 = axes[1]
    ax2.hist(errores, bins=30, edgecolor='black', alpha=0.7)
    ax2.axvline(x=0, color='red', linestyle='--', lw=2, label='Error = 0')
    ax2.axvline(x=errores.mean(), color='green', linestyle='-', lw=2, label=f'Media = {errores.mean():.2f}')
    ax2.set_xlabel('Error (Imputado - Original)', fontsize=11)
    ax2.set_ylabel('Frecuencia', fontsize=11)
    ax2.set_title('Distribución de Errores', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # 3. Residuales
    ax3 = axes[2]
    ax3.scatter(valores_imputados, errores, alpha=0.6, s=50)
    ax3.axhline(y=0, color='red', linestyle='--', lw=2)
    ax3.set_xlabel('Valores Imputados', fontsize=11)
    ax3.set_ylabel('Residuales', fontsize=11)
    ax3.set_title('Análisis de Residuales', fontweight='bold')
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

def crear_heatmap_metricas(df_comparacion):
    """
    Crea un heatmap de las métricas por método.
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Heatmap de Métricas por Método', fontsize=16, fontweight='bold')

    metricas = ['MAE', 'RMSE', 'MAPE (%)', 'R²']

    for idx, metrica in enumerate(metricas):
        ax = axes[idx // 2, idx % 2]
        pivot = df_comparacion.pivot_table(values=metrica, index='Columna', columns='Método')

        # Para R², mayor es mejor (usar cmap inverso)
        if metrica == 'R²':
            sns.heatmap(pivot, annot=True, fmt='.3f', cmap='RdYlGn', ax=ax,
                       cbar_kws={'label': metrica}, vmin=0, vmax=1)
        else:
            sns.heatmap(pivot, annot=True, fmt='.3f', cmap='RdYlGn_r', ax=ax,
                       cbar_kws={'label': metrica})

        ax.set_title(f'{metrica}', fontweight='bold')
        ax.set_xlabel('Método')
        ax.set_ylabel('Columna')

    plt.tight_layout()
    plt.show()

# 📋 PARTE 3: Carga de nuestros datos

## 3.1 Cargar Datos

In [ ]:
# Cargar tu dataset
df_tus_datos = pd.read_csv('/content/retail_store_sales.csv')
# df_tus_datos = pd.read_excel('tu_archivo.xlsx')

# Para este ejemplo, usaremos el dataset que creamos
# df_tus_datos = df.copy()

# Introducir algunos nulos artificialmente para demostrar la limpieza
# np.random.seed(123)
# indices_nulos = np.random.choice(df_tus_datos.index, 80, replace=False)
# df_tus_datos.loc[indices_nulos[:30], 'edad'] = np.nan
# df_tus_datos.loc[indices_nulos[30:60], 'salario'] = np.nan
# df_tus_datos.loc[indices_nulos[60:], 'departamento'] = np.nan

print("✅ Datos cargados")
print(f"Dimensiones: {df_tus_datos.shape}")
# print(f"\nNulos por columna:")
# print(df_tus_datos.isna().sum())

✅ Datos cargados
Dimensiones: (12575, 11)


## 3.2 Análisis Inicial

In [ ]:
# Analizar tus datos
analizador = AnalizadorDatos(df_tus_datos)
reporte_tus_datos = analizador.generar_reporte_completo()


🔍 ANÁLISIS DE TIPOS DE VARIABLES

📊 FECHAS: 11
   • Transaction ID: Únicos=12575, Nulos=0 (0.0%)
   • Customer ID: Únicos=25, Nulos=0 (0.0%)
   • Category: Únicos=8, Nulos=0 (0.0%)
   • Item: Únicos=200, Nulos=1213 (9.6%)
   • Price Per Unit: Únicos=25, Nulos=609 (4.8%), Rango=[5.00, 41.00]
   • Quantity: Únicos=10, Nulos=604 (4.8%), Rango=[1.00, 10.00]
   • Total Spent: Únicos=227, Nulos=604 (4.8%), Rango=[5.00, 410.00]
   • Payment Method: Únicos=3, Nulos=0 (0.0%)
   • Location: Únicos=2, Nulos=0 (0.0%)
   • Transaction Date: Únicos=1114, Nulos=0 (0.0%)
   • Discount Applied: Únicos=2, Nulos=4199 (33.4%)

📉 ANÁLISIS DE DATOS FALTANTES

📋 Resumen de Datos Faltantes:
         columna  n_nulos  pct_nulos              patron   tipo
Discount Applied     4199  33.391650 MCAR (posiblemente) fechas
            Item     1213   9.646123 MCAR (posiblemente) fechas
  Price Per Unit      609   4.842942 MCAR (posiblemente) fechas
        Quantity      604   4.803181 MCAR (posiblemente) fechas
   

## 3.3 Comparar Preservación de Datos por Método

Compara cuántos datos se preservan con diferentes estrategias de imputación.

In [ ]:
# Comparar diferentes estrategias para ver cuál preserva más datos

print("\n" + "="*70)
print("🔍 COMPARACIÓN DE PRESERVACIÓN DE DATOS")
print("="*70)

# Identificar variables
analizador_comp = AnalizadorDatos(df_tus_datos)
clasificacion_comp = analizador_comp.identificar_tipos_variables()

cols_num = (clasificacion_comp['numericas_continuas'] +
           clasificacion_comp['numericas_discretas'])
cols_num_nulos = [c for c in cols_num if df_tus_datos[c].isna().sum() > 0]

estrategias = [
    {'nombre': 'Sin imputar (solo eliminar)', 'metodo': None},
    {'nombre': 'KNN (k=3)', 'metodo': 'knn', 'params': {'n_neighbors': 3}},
    {'nombre': 'KNN (k=5)', 'metodo': 'knn', 'params': {'n_neighbors': 5}},
    {'nombre': 'KNN (k=7)', 'metodo': 'knn', 'params': {'n_neighbors': 7}},
    {'nombre': 'MICE', 'metodo': 'mice', 'params': {'max_iter': 10}},
    {'nombre': 'Regresión', 'metodo': 'regresion', 'params': {}},
]

resultados_preservacion = []

for estrategia in estrategias:
    df_temp = df_tus_datos.copy()

    try:
        # Aplicar imputación si corresponde
        if estrategia['metodo'] and len(cols_num_nulos) > 0:
            imputador_temp = ImputadorAvanzado(df_temp)

            if estrategia['metodo'] == 'knn':
                df_temp = imputador_temp.imputar_knn(
                    cols_num_nulos,
                    n_neighbors=estrategia['params']['n_neighbors']
                )
            elif estrategia['metodo'] == 'mice':
                df_temp = imputador_temp.imputar_mice(
                    cols_num_nulos,
                    max_iter=estrategia['params']['max_iter']
                )
            elif estrategia['metodo'] == 'regresion':
                df_temp = imputador_temp.imputar_regresion(cols_num_nulos)

        # Eliminar nulos categóricos
        imputador_temp = ImputadorAvanzado(df_temp)
        df_temp = imputador_temp.eliminar_categoricos_nulos(umbral_porcentaje=50.0)

        # Calcular preservación
        filas_preservadas = len(df_temp)
        pct_preservado = (filas_preservadas / len(df_tus_datos)) * 100
        nulos_restantes = df_temp.isna().sum().sum()

        resultados_preservacion.append({
            'Estrategia': estrategia['nombre'],
            'Filas Finales': filas_preservadas,
            '% Preservado': f"{pct_preservado:.1f}%",
            'Nulos Restantes': nulos_restantes,
            'Filas Perdidas': len(df_tus_datos) - filas_preservadas
        })

    except Exception as e:
        print(f"⚠️ Error con {estrategia['nombre']}: {str(e)}")

# Mostrar resultados
df_preservacion = pd.DataFrame(resultados_preservacion)
df_preservacion = df_preservacion.sort_values('Filas Finales', ascending=False)

print("\n" + "="*70)
print("📊 TABLA COMPARATIVA DE PRESERVACIÓN")
print("="*70)
print(df_preservacion.to_string(index=False))

# Recomendación
mejor = df_preservacion.iloc[0]
print("\n" + "="*70)
print("🏆 RECOMENDACIÓN")
print("="*70)
print(f"\n✅ Mejor estrategia: {mejor['Estrategia']}")
print(f"   📈 Preserva: {mejor['% Preservado']} de los datos")
print(f"   📉 Nulos restantes: {mejor['Nulos Restantes']}")
print(f"\n💡 Usa este método en la sección 4.4 para obtener mejores resultados")


🔍 COMPARACIÓN DE PRESERVACIÓN DE DATOS

🔍 ANÁLISIS DE TIPOS DE VARIABLES

📊 FECHAS: 11
   • Transaction ID: Únicos=12575, Nulos=0 (0.0%)
   • Customer ID: Únicos=25, Nulos=0 (0.0%)
   • Category: Únicos=8, Nulos=0 (0.0%)
   • Item: Únicos=200, Nulos=1213 (9.6%)
   • Price Per Unit: Únicos=25, Nulos=609 (4.8%), Rango=[5.00, 41.00]
   • Quantity: Únicos=10, Nulos=604 (4.8%), Rango=[1.00, 10.00]
   • Total Spent: Únicos=227, Nulos=604 (4.8%), Rango=[5.00, 410.00]
   • Payment Method: Únicos=3, Nulos=0 (0.0%)
   • Location: Únicos=2, Nulos=0 (0.0%)
   • Transaction Date: Únicos=1114, Nulos=0 (0.0%)
   • Discount Applied: Únicos=2, Nulos=4199 (33.4%)

🗑️  ELIMINACIÓN DE NULOS EN VARIABLES CATEGÓRICAS
✅ 'Item': 1213 filas eliminadas (9.6% nulos)
✅ 'Discount Applied': 3783 filas eliminadas (33.3% nulos)

📊 Filas totales eliminadas: 4996 (39.7%)

🗑️  ELIMINACIÓN DE NULOS EN VARIABLES CATEGÓRICAS
✅ 'Item': 1213 filas eliminadas (9.6% nulos)
✅ 'Discount Applied': 3783 filas eliminadas (33.3% nu

## 3.4 Pipeline de Limpieza Paso a Paso

**Estrategia**:
1. Primero **imputamos** las variables numéricas (no perdemos filas)
2. Luego **eliminamos** nulos categóricos (perdemos menos filas)
3. Finalmente corregimos fechas

In [ ]:
# ═══════════════════════════════════════════════════════════
# PASO 1: Identificar variables numéricas y categóricas
# ═══════════════════════════════════════════════════════════

print("\n" + "="*70)
print("PASO 1: IDENTIFICANDO TIPOS DE VARIABLES")
print("="*70)

analizador_temp = AnalizadorDatos(df_tus_datos)
clasificacion = analizador_temp.identificar_tipos_variables()

# Variables numéricas
cols_numericas = (clasificacion['numericas_continuas'] +
                 clasificacion['numericas_discretas'])
cols_numericas_con_nulos = [col for col in cols_numericas
                            if df_tus_datos[col].isna().sum() > 0]

# Variables categóricas
cols_categoricas = (clasificacion['categoricas_nominales'] +
                   clasificacion['categoricas_ordinales'])
cols_categoricas_con_nulos = [col for col in cols_categoricas
                             if df_tus_datos[col].isna().sum() > 0]

print(f"\n📊 Variables numéricas con nulos: {len(cols_numericas_con_nulos)}")
print(f"📊 Variables categóricas con nulos: {len(cols_categoricas_con_nulos)}")

# Guardar estado inicial
nulos_iniciales = df_tus_datos.isna().sum().sum()
filas_iniciales = len(df_tus_datos)

print(f"\n🔸 Estado inicial:")
print(f"   Filas: {filas_iniciales:,}")
print(f"   Nulos totales: {nulos_iniciales:,}")


PASO 1: IDENTIFICANDO TIPOS DE VARIABLES

🔍 ANÁLISIS DE TIPOS DE VARIABLES

📊 FECHAS: 11
   • Transaction ID: Únicos=12575, Nulos=0 (0.0%)
   • Customer ID: Únicos=25, Nulos=0 (0.0%)
   • Category: Únicos=8, Nulos=0 (0.0%)
   • Item: Únicos=200, Nulos=1213 (9.6%)
   • Price Per Unit: Únicos=25, Nulos=609 (4.8%), Rango=[5.00, 41.00]
   • Quantity: Únicos=10, Nulos=604 (4.8%), Rango=[1.00, 10.00]
   • Total Spent: Únicos=227, Nulos=604 (4.8%), Rango=[5.00, 410.00]
   • Payment Method: Únicos=3, Nulos=0 (0.0%)
   • Location: Únicos=2, Nulos=0 (0.0%)
   • Transaction Date: Únicos=1114, Nulos=0 (0.0%)
   • Discount Applied: Únicos=2, Nulos=4199 (33.4%)

📊 Variables numéricas con nulos: 0
📊 Variables categóricas con nulos: 0

🔸 Estado inicial:
   Filas: 12,575
   Nulos totales: 7,229


In [ ]:
# ═══════════════════════════════════════════════════════════
# PASO 2: IMPUTAR VARIABLES NUMÉRICAS (preserva todas las filas)
# ═══════════════════════════════════════════════════════════

print("\n" + "="*70)
print("PASO 2: IMPUTANDO VARIABLES NUMÉRICAS")
print("="*70)

df_con_numericas_imputadas = df_tus_datos.copy()

if len(cols_numericas_con_nulos) > 0:
    imputador_numerico = ImputadorAvanzado(df_con_numericas_imputadas)

    # Selecciona el mejor método basado en la comparación anterior
    # o usa 'knn' como default (generalmente el más balanceado)
    df_con_numericas_imputadas = imputador_numerico.imputar_knn(
        columnas=cols_numericas_con_nulos,
        n_neighbors=5
    )

    # Alternativas (descomenta la que prefieras):
    # df_con_numericas_imputadas = imputador_numerico.imputar_mice(
    #     columnas=cols_numericas_con_nulos, max_iter=10)
    # df_con_numericas_imputadas = imputador_numerico.imputar_regresion(
    #     columnas=cols_numericas_con_nulos)
    # df_con_numericas_imputadas = imputador_numerico.imputar_interpolacion(
    #     columnas=cols_numericas_con_nulos, metodo='linear')

    print(f"\n✅ Variables numéricas imputadas exitosamente")
    print(f"✅ Filas preservadas: {len(df_con_numericas_imputadas):,} (100%)")
else:
    print("\n✅ No hay variables numéricas con nulos")

# Verificar nulos restantes en numéricas
nulos_numericos_restantes = df_con_numericas_imputadas[cols_numericas].isna().sum().sum()
print(f"\n📊 Nulos en variables numéricas: {nulos_numericos_restantes}")


PASO 2: IMPUTANDO VARIABLES NUMÉRICAS

✅ No hay variables numéricas con nulos

📊 Nulos en variables numéricas: 0.0


In [ ]:
# ═══════════════════════════════════════════════════════════
# PASO 3: ELIMINAR NULOS CATEGÓRICOS (puede eliminar filas)
# ═══════════════════════════════════════════════════════════

print("\n" + "="*70)
print("PASO 3: LIMPIANDO VARIABLES CATEGÓRICAS")
print("="*70)

df_limpio = df_con_numericas_imputadas.copy()

if len(cols_categoricas_con_nulos) > 0:
    print(f"\n🔸 Variables categóricas con nulos: {cols_categoricas_con_nulos}")
    print(f"\nNulos por variable categórica:")
    for col in cols_categoricas_con_nulos:
        n_nulos = df_limpio[col].isna().sum()
        pct = (n_nulos / len(df_limpio)) * 100
        print(f"   • {col}: {n_nulos} ({pct:.1f}%)")

    # Aplicar limpieza de categóricas
    imputador_categorico = ImputadorAvanzado(df_limpio)

    # Ajusta el umbral según tus necesidades:
    # - umbral bajo (30-40%): Más estricto, elimina más columnas
    # - umbral medio (50%): Balanceado (default)
    # - umbral alto (70-80%): Más permisivo, conserva más columnas
    df_limpio = imputador_categorico.eliminar_categoricos_nulos(
        umbral_porcentaje=50.0  # Ajusta este valor según tus necesidades
    )
else:
    print("\n✅ No hay variables categóricas con nulos")

filas_despues_categoricas = len(df_limpio)
filas_eliminadas = filas_iniciales - filas_despues_categoricas
pct_eliminado = (filas_eliminadas / filas_iniciales) * 100

print(f"\n📊 Impacto de limpieza categórica:")
print(f"   Filas eliminadas: {filas_eliminadas:,} ({pct_eliminado:.1f}%)")
print(f"   Filas preservadas: {filas_despues_categoricas:,} ({100-pct_eliminado:.1f}%)")


PASO 3: LIMPIANDO VARIABLES CATEGÓRICAS

✅ No hay variables categóricas con nulos

📊 Impacto de limpieza categórica:
   Filas eliminadas: 0 (0.0%)
   Filas preservadas: 12,575 (100.0%)


In [ ]:
# ═══════════════════════════════════════════════════════════
# PASO 4: CORREGIR FECHAS (opcional)
# ═══════════════════════════════════════════════════════════

print("\n" + "="*70)
print("PASO 4: CORRIGIENDO FORMATOS DE FECHA")
print("="*70)

if len(clasificacion['fechas']) > 0:
    corrector = CorreccionFormatos(df_limpio)
    df_limpio = corrector.corregir_fechas()
    print("\n✅ Fechas corregidas")
else:
    print("\n✅ No hay columnas de fecha para corregir")


PASO 4: CORRIGIENDO FORMATOS DE FECHA

📅 CORRECCIÓN DE FORMATOS DE FECHA
✅ 'Transaction Date': 12575 fechas convertidas, 0 fallos
   Rango: 2022-01-01 00:00:00 a 2025-01-18 00:00:00

✅ Fechas corregidas


In [ ]:
# ═══════════════════════════════════════════════════════════
# RESUMEN FINAL DEL PROCESO
# ═══════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📊 RESUMEN FINAL DE LIMPIEZA")
print("="*70)

nulos_finales = df_limpio.isna().sum().sum()
filas_finales = len(df_limpio)

print(f"\n🔸 DATOS ORIGINALES:")
print(f"   Filas: {filas_iniciales:,}")
print(f"   Nulos: {nulos_iniciales:,}")

print(f"\n🔸 DATOS LIMPIOS:")
print(f"   Filas: {filas_finales:,}")
print(f"   Nulos: {nulos_finales:,}")

print(f"\n🔸 CAMBIOS:")
print(f"   Filas eliminadas: {filas_iniciales - filas_finales:,} ({((filas_iniciales - filas_finales) / filas_iniciales * 100):.1f}%)")
print(f"   Filas preservadas: {filas_finales:,} ({(filas_finales / filas_iniciales * 100):.1f}%)")
print(f"   Reducción de nulos: {nulos_iniciales - nulos_finales:,} ({((nulos_iniciales - nulos_finales) / max(nulos_iniciales, 1) * 100):.1f}%)")

print(f"\n🔸 NULOS POR COLUMNA (final):")
nulos_por_col = df_limpio.isna().sum()
if nulos_por_col.sum() > 0:
    print(nulos_por_col[nulos_por_col > 0])
else:
    print("   ✅ No hay nulos restantes")

print("\n" + "="*70)
print("✅ LIMPIEZA COMPLETADA EXITOSAMENTE")
print("="*70)


📊 RESUMEN FINAL DE LIMPIEZA

🔸 DATOS ORIGINALES:
   Filas: 12,575
   Nulos: 7,229

🔸 DATOS LIMPIOS:
   Filas: 12,575
   Nulos: 7,229

🔸 CAMBIOS:
   Filas eliminadas: 0 (0.0%)
   Filas preservadas: 12,575 (100.0%)
   Reducción de nulos: 0 (0.0%)

🔸 NULOS POR COLUMNA (final):
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Discount Applied    4199
dtype: int64

✅ LIMPIEZA COMPLETADA EXITOSAMENTE


## 3.4b Alternativa: Pipeline Automático con Control de Imputación (Opcional)

In [ ]:
# Solo ejecuta esta celda si NO ejecutaste la sección 4.4 paso a paso

# Crear una copia del dataset original
df_para_pipeline = df_tus_datos.copy()

# Crear limpiador
limpiador_auto = LimpiadorCompleto(df_para_pipeline)

# Pipeline automático
# IMPORTANTE: El pipeline PRIMERO imputa numéricas, LUEGO elimina categóricas
df_limpio_auto = limpiador_auto.pipeline_completo(
    metodo_imputacion='knn',     # Opciones: 'knn', 'mice', 'regresion', 'interpolacion'
    n_neighbors=5,                # Para KNN: número de vecinos
    # max_iter=10,                # Para MICE: número de iteraciones
    # metodo_interpolacion='linear', # Para interpolación: tipo de interpolación
    eliminar_nulos_categoricos=True,  # True = elimina filas con nulos categóricos
    corregir_fechas=True,             # True = corrige formatos de fecha
    umbral_categoricos=50.0           # % máximo de nulos antes de eliminar columna
)

print("\n" + "="*70)
print("✅ PIPELINE AUTOMÁTICO COMPLETADO")
print("="*70)
print(f"\nEl dataset limpio está en la variable: df_limpio_auto")
print(f"Puedes usar df_limpio = df_limpio_auto.copy() para continuar")


🚀 INICIANDO PIPELINE DE LIMPIEZA COMPLETO

🔍 ANÁLISIS DE TIPOS DE VARIABLES

📊 FECHAS: 11
   • Transaction ID: Únicos=12575, Nulos=0 (0.0%)
   • Customer ID: Únicos=25, Nulos=0 (0.0%)
   • Category: Únicos=8, Nulos=0 (0.0%)
   • Item: Únicos=200, Nulos=1213 (9.6%)
   • Price Per Unit: Únicos=25, Nulos=609 (4.8%), Rango=[5.00, 41.00]
   • Quantity: Únicos=10, Nulos=604 (4.8%), Rango=[1.00, 10.00]
   • Total Spent: Únicos=227, Nulos=604 (4.8%), Rango=[5.00, 410.00]
   • Payment Method: Únicos=3, Nulos=0 (0.0%)
   • Location: Únicos=2, Nulos=0 (0.0%)
   • Transaction Date: Únicos=1114, Nulos=0 (0.0%)
   • Discount Applied: Únicos=2, Nulos=4199 (33.4%)

📉 ANÁLISIS DE DATOS FALTANTES

📋 Resumen de Datos Faltantes:
         columna  n_nulos  pct_nulos              patron   tipo
Discount Applied     4199  33.391650 MCAR (posiblemente) fechas
            Item     1213   9.646123 MCAR (posiblemente) fechas
  Price Per Unit      609   4.842942 MCAR (posiblemente) fechas
        Quantity      60

### 3.5 Validar Resultados

In [ ]:
print("\n" + "="*70)
print("✅ VALIDACIÓN DE LIMPIEZA")
print("="*70)

print(f"\nFilas antes: {len(df_tus_datos)}")
print(f"Filas después: {len(df_limpio)}")
print(f"Filas eliminadas: {len(df_tus_datos) - len(df_limpio)}")

print(f"\nNulos antes: {df_tus_datos.isna().sum().sum()}")
print(f"Nulos después: {df_limpio.isna().sum().sum()}")

print(f"\n📊 Nulos por columna (después):")
print(df_limpio.isna().sum())


✅ VALIDACIÓN DE LIMPIEZA

Filas antes: 12575
Filas después: 12575
Filas eliminadas: 0

Nulos antes: 7229
Nulos después: 7229

📊 Nulos por columna (después):
Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64


## 3.6 Guardar Datos Limpios

In [ ]:
# Guardar datos limpios
df_limpio.to_csv('datos_limpios.csv', index=False)
# df_limpio.to_excel('datos_limpios.xlsx', index=False)

print("✅ Datos limpios guardados (descomenta las líneas de arriba para guardar)")

✅ Datos limpios guardados (descomenta las líneas de arriba para guardar)
